In [1]:
#from atmos_models import LISA
import open3d as o3d
# import struct
import numpy as np
import os

In [2]:
from tqdm.notebook import tqdm as tqdm_nb


In [3]:
bin_path ='/home-local2/vikon4.extra.nobkp/mmdetection3d/data/kitti/training/velodyne'
fname =  '000015.bin'

In [4]:
raw_lidar = np.fromfile(os.path.join(bin_path, fname), dtype=np.float32).reshape((-1, 4))
this_pcd = o3d.geometry.PointCloud()
this_pcd.points = o3d.utility.Vector3dVector(raw_lidar[:,:3])
# this_pcd.colors = this_pcd.points  
#o3d.io.write_point_cloud(os.path.join(, some_pcd)


In [5]:
def bin_to_pcd(raw_lidar):
    this_pcd = o3d.geometry.PointCloud()
    this_pcd.points = o3d.utility.Vector3dVector(raw_lidar[:,:3])
    return this_pcd

In [6]:
def draw_pcd(pcd, where='mat_3d'):
    if where == 'opn_nb':
        visualizer = o3d.JVisualizer()
        visualizer.add_geometry(pcd)
        visualizer.show()
    elif where == 'opn_view':
        web_visualizer.draw([pcd])
    elif where == 'mat_3d':
        plt.figure()
        pts = np.asarray(pcd.points)
        plt.scatter(pts[:,0], pts[:,1], pts[:,2])
        plt.grid()
        plt.show()
    elif where == 'mat_2d':
        plt.figure()
        pts = np.asarray(pcd.points)
        plt.scatter(pts[:,0], pts[:,1])
        plt.grid()
        plt.show()

In [7]:
draw_pcd(this_pcd, "opn_nb")

A Jupyter Widget

In [8]:
from atmos_models import LISA #install PyMIeScatt using conda

In [9]:
lisa = LISA(atm_model='rain')

'''
        Augment clean pointcloud for a given rain rate
        Parameters
        ----------
        pc : pointcloud (N,4) -> x,y,z,reflectivity
        Rr : rain rate (mm/hr)
        Returns
        -------
        pc_new : new noisy point cloud (N,5) -> x,y,z,reflectivity,label
                        label 0 -> lost point
                        label 1 -> randomly scattered point
                        label 2 -> not-scattered 
'''

In [10]:
def add_noisy_weather_color(noisy_pcd, raw_data):
    """the color code expects input of shape [-1,5]. 
    The output of LISA simulation """

    color = np.zeros(shape=np.asarray(noisy_pcd.points).shape)
    random_scatter_idx = np.where(raw_data[:,4] == 1) #randonmly scattered points
    lost_point_idx  = np.where(raw_data[:,4] == 0) #lost points 

    color[random_scatter_idx, 1] = 1 # 0, 1, 0
    color[lost_point_idx, 0]     = 1 # 1, 0, 0

    noisy_pcd.colors =  o3d.utility.Vector3dVector(color)
    return noisy_pcd

In [11]:
data_sim  = lisa.augment(raw_lidar,55) #snow_rate_75mm/hr


In [12]:
noisy_pcd = bin_to_pcd(data_sim)

In [13]:
noisy_pcd = add_noisy_weather_color(noisy_pcd, data_sim)

In [14]:
lisa = LISA(atm_model='snow')
data_sim  = lisa.augment(raw_lidar,55) #snow_rate_75mm/hr
noisy_pcd = bin_to_pcd(data_sim)
noisy_pcd = add_noisy_weather_color(noisy_pcd, data_sim)
draw_pcd(noisy_pcd, "opn_nb")

A Jupyter Widget

In [15]:
lisa = LISA(atm_model='snow')
data_sim  = lisa.augment(raw_lidar,150) #snow_rate_75mm/hr
noisy_pcd = bin_to_pcd(data_sim)
noisy_pcd = add_noisy_weather_color(noisy_pcd, data_sim)
draw_pcd(noisy_pcd, "opn_nb")

A Jupyter Widget